In [ ]:
%matplotlib ipympl
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexPrograms

# Shortest path

In [ ]:
graph = GraphOfConvexPrograms()

n = 3
for i in range(n):
    for j in range(n):
        v = graph.add_vertex((i,j))
        x = v.add_variable(2)
        c = np.array([i, j])
        v.add_constraint(cp.norm(x - c, 2) <= .3)

In [ ]:
graph.edges = []
edges = [
    ((0, 0), (1, 0)),
    ((1, 0), (2, 0)),
    ((2, 0), (2, 1)),
    ((0, 1), (0, 0)),
    ((0, 1), (0, 2)),
    ((1, 1), (1, 0)),
    ((1, 1), (0, 1)),
    ((1, 1), (1, 2)),
    ((2, 1), (1, 1)),
    ((2, 1), (2, 2)),
    ((1, 2), (0, 2)),
    ((1, 2), (2, 2)),
        ]
for tail_name, head_name in edges:
    tail = graph.get_vertex(tail_name)
    head = graph.get_vertex(head_name)
    edge = graph.add_edge(tail, head)
    edge.add_cost(cp.norm(head.variables[0] - tail.variables[0], 2))

In [ ]:
source = graph.vertices[0]
target = graph.vertices[2]
prob = graph.solve_shortest_path(source, target)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))
plt.gca().set_aspect('equal')
plt.axis('off')
graph.plot_2d()
graph.plot_subgraph_2d()
# plt.savefig('shortest_path.pdf')

# Traveling salesman (DFJ)

In [ ]:
gcs.edges = []
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                dx = abs(i - k)
                dy = abs(j - l)
                if dx <= 1 and dy <= 1 and dx + dy != 0:
                    tail = gcs.get_vertex_by_name(str((i, j)))
                    head = gcs.get_vertex_by_name(str((k, l)))
                    edge = gcs.add_edge(tail, head)
                    edge.add_cost(cp.norm(head.variables[0] - tail.variables[0], 2))

In [ ]:
prob = gcs.solve_traveling_salesman()
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
# plt.savefig('traveling_salesman.pdf')

# Traveling salesman (MTZ)

In [ ]:
gcs = GraphOfConvexSets()

for i in range(n):
    for j in range(n):
        v = gcs.add_vertex(str((i,j)))
        x = v.add_variable(2)
        o = v.add_variable(1)
        c = np.array([i, j])
        v.add_constraint(cp.norm(x - c, 2) <= .3)
        if i == j == 0:
            v.add_constraint(o == 1)
        else:
            v.add_constraint(o >= 1)
            v.add_constraint(o <= n ** 2)

In [ ]:
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                dx = abs(i - k)
                dy = abs(j - l)
                if dx <= 1 and dy <= 1 and dx + dy != 0:
                    tail = gcs.get_vertex_by_name(str((i, j)))
                    head = gcs.get_vertex_by_name(str((k, l)))
                    edge = gcs.add_edge(tail, head)
                    edge.add_cost(cp.norm(head.variables[0] - tail.variables[0], 2))
                    if head.name != "(0, 0)":
                        edge.add_constraint(head.variables[1] == tail.variables[1] + 1)

In [ ]:
prob = gcs.solve_traveling_salesman(subtour_elimination=False)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

# Facility location

In [ ]:
gcs = GraphOfConvexSets()

facilities = [1, 3, 5, 7]
for i in range(n):
    for j in range(n):
        norm = np.inf if gcs.num_vertices() in facilities else 2
        v = gcs.add_vertex(str((i,j)))
        x = v.add_variable(2)
        c = np.array([i, j])
        v.add_constraint(cp.norm(x - c, norm) <= .25)

In [ ]:
gcs.edges = []
customers = [i for i in range(gcs.num_vertices()) if i not in facilities]
for facility in facilities:
    for customer in customers:
        tail = gcs.vertices[facility]
        head = gcs.vertices[customer]
        edge = gcs.add_edge(tail, head)
        edge.add_cost(cp.norm(tail.variables[0] - head.variables[0], 2))

In [ ]:
prob = gcs.solve_facility_location(facilities)
print('Problem status:', prob.status)
print('Optimal value:', prob.value)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4, 4))
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
# plt.savefig('facility_location.pdf')